In [1]:
import numpy as np
import pandas as pd

In [2]:
pip install -U spacy

Requirement already up-to-date: spacy in c:\users\rupal\anaconda3\lib\site-packages (2.2.4)
Note: you may need to restart the kernel to use updated packages.


In [3]:
!python -m spacy download en_core_web_md

[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [4]:
# read the data into a pandas dataframe
df=pd.read_csv('twitterMining.csv')
df.sample(frac=0.005)

,Unnamed: 0,user,location,Tweets,Subjectivity,Polarity,Positive,Negative,Neutral,compound,TextBlobAnalysis,VADERAnalysis,TextBlobAnalysis_Val,VADERAnalysis_Val
406,406,FWolfenbuttel,"Hannover, Deutschland",Jojo Due to the Corona Virus the stalk market ...,0.3375,-0.06250,0.000,0.149,0.851,-0.2732,Negative,Negative,0,0
665,665,aClassicLiberal,Digital Nomad (Tampa Bay),This is for Italy Ive a long thread on the Co...,0.4125,0.14375,0.126,0.000,0.874,0.3182,Positive,Positive,2,2
327,327,PresidentKovach,NaN,Im out yall she got me before corona could lma...,0.0000,0.00000,0.000,0.000,1.000,0.0000,Neutral,Neutral,1,1
669,669,httpsjhey,Jamaica,Me in 2050 telling my grandkids about Corona V...,0.0000,0.00000,0.000,0.000,1.000,0.0000,Neutral,Neutral,1,1
971,971,worom_z,"Federal Capital Territory, Nig",I just noticed corona virus has emoji,0.0000,0.00000,0.000,0.000,1.000,0.0000,Neutral,Neutral,1,1


In [5]:
# setup the data for VADERAnalysis_Val
X, y = df['Tweets'], df['VADERAnalysis_Val']

from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=1)

Xtrain = Xtrain.copy()
Xtest = Xtest.copy()
ytrain = ytrain.copy()
ytest = ytest.copy()

In [6]:
def custom_tokenizer(doc):

    # use spacy to filter out noise
    tokens = [token.lemma_.lower() 
                        for token in doc 
                               if (
                                    len(token) >= 2 and # only preserve tokens that are greater than 2 characters long
                                    token.pos_ in ['PROPN', 'NOUN', 'ADJ', 'VERB', 'ADV'] and # only preserve selected pos
                                    #token.text in nlp.vocab and # check if token in vocab 
                                    token.is_alpha and # only preserve tokens that are fully alpha (not numeric or alpha-numeric)
                                    #not token.is_digit and # get rid of tokens that are fully numeric
                                    not token.is_punct and # get rid of tokens that are punctuations
                                    not token.is_space and # get rid of tokens that are spaces
                                    not token.is_stop and # get rid of tokens that are stop words
                                    not token.is_currency # get rid of tokens that denote currencies
                                )
                   ]

    # return cleaned-up text
    return ' '.join(tokens)

In [7]:
import spacy
nlp = spacy.load("en_core_web_md", disable=['parser', 'ner'])
corpus = nlp.pipe(Xtrain)
clean_corpus = [custom_tokenizer(doc) for doc in corpus]
Xtrain = pd.Series(clean_corpus)
Xtrain.head()


corpus = nlp.pipe(Xtest)
clean_corpus = [custom_tokenizer(doc) for doc in corpus]
Xtest = pd.Series(clean_corpus)
Xtest.head()

0                                         toppe corona
1    trend continue rare day march delhi well weath...
2     crush agree date play away mr oxygen problem don
3          balanced view forget check study retired dr
4                              guys corona invade hell
dtype: object

In [8]:
# setup the preprocessing->model pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

clf = Pipeline(steps=[
    ('tfidf', TfidfVectorizer(
        binary=False,
        use_idf=True, smooth_idf=True, # idf  - with smoothing
        norm='l2', # tfidf - l2 norm
        lowercase=True, stop_words='english', 
        #token_pattern='(?u)\\b\\w\\w+\\b', 
        min_df=1, max_df=1.0, max_features=None, 
        ngram_range=(1, 1)
    )),
    ('nb', MultinomialNB(
        fit_prior=True, # learn class prior-probabilities from data
        class_prior=None # none - go with whatever fit-prior says
    ))])


In [9]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'tfidf__sublinear_tf':['True','False'], 
   'nb__alpha': [0.00002, 0.000002, 0.002, 0.0000002,0.2, 0.1,0.001]}

gscv = GridSearchCV(clf, param_grid, cv=4, return_train_score = False)

In [10]:
gscv.fit(Xtrain, ytrain)

GridSearchCV(cv=4, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

In [11]:
# Uncomment below to see the results of hyperparameter tuning
print ("-"*100)
print(gscv.best_estimator_, "\n")
print ("-"*100)
print(gscv.best_score_, "\n")
print ("-"*100)
print(gscv.best_params_, "\n")
print ("-"*100)
print(gscv.cv_results_, "\n")
print ("-"*100)

----------------------------------------------------------------------------------------------------
Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf='True',
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('nb',
                 MultinomialNB(alpha=0.2,

In [12]:
from sklearn import metrics
ypred = gscv.best_estimator_.predict(Xtest)

from sklearn import metrics
print (metrics.accuracy_score(ytest, ypred))
print("confusion matrix")
print (metrics.confusion_matrix(ytest, ypred))
print (metrics.classification_report(ytest, ypred))
# TN,FP,FN,TP = metrics.confusion_matrix(y_true=ytest, y_pred=ypred).ravel()

0.475
confusion matrix
[[39 14 14]
 [21 33 15]
 [29 12 23]]
              precision    recall  f1-score   support

           0       0.44      0.58      0.50        67
           1       0.56      0.48      0.52        69
           2       0.44      0.36      0.40        64

    accuracy                           0.48       200
   macro avg       0.48      0.47      0.47       200
weighted avg       0.48      0.47      0.47       200

